## Import packages:

In [88]:
from PIL import Image, ImageOps
import numpy as np
from os import listdir

from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from keras.optimizers import Adam

from bc_utils import *

#from keras.layers.core import Dense, Dropout, Flatten
#from keras.layers.convolutional import MaxPooling2D, AveragePooling2D

# Load data:

In [200]:
def load_data(nr_samples = 438, dim = (384, 480), folder_path = 'priors_all_copy'):
    input_images = []
    target_pixels = []
    passed_pix = 0
    passed_img = 0
    #dim = (192, 240) #width, heigth (ratio 0.8) 1920, 2400
    for filename in sorted(listdir(folder_path)):
        if "mask" in filename:
            img_data = Image.open(folder_path + '/' + filename)
            data = np.array(img_data)
            pixel_coord = np.where(data == True)
            img_size = img_data.size
            target_pixels.append([filename, pixel_coord[0][0]/img_size[0],pixel_coord[1][0]/img_size[1]])
        elif "DS_Store" in filename:
            pass
        else:
            try:
                img_data = Image.open(folder_path + '/' + filename)
                img_rgb= img_data.convert('RGB')
                # resize image
                img_resized = img_rgb.resize(dim)
                data = np.array(img_resized)
                input_images.append([filename,data])
            except (UnidentifiedImageError, NameError):
                passed_img += 1

    input_images = np.asarray(input_images)
    target_pixels = np.asarray(target_pixels)
    
    return input_images, target_pixels

#print(input_images.size)
#print(len(target_pixels))
#print(target_pixels)
#print(input_images)
#print(passed_pix)
#print(passed_img)

In [199]:
#test PIL

from PIL import Image

print("Image info:")
# load the image
image = Image.open('68344674535469533878377A584E43593337513070413D3D_537153536F422F464D67357430426C43547431456F774455364A6367436A4C48_20110406_4.png')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)

print()
print("Mask info:")
# load the image
image = Image.open('68344674535469533878377A584E43593337513070413D3D_537153536F422F464D67357430426C43547431456F774455364A6367436A4C48_20110406_4_mask.png')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
img_grey= image.convert('RGB')
print(img_grey.mode)


Image info:
PNG
RGBA
(1600, 2400)

Mask info:
PNG
RGBA
(1600, 2400)
RGB


In [40]:
#manual data augmentation (rotation)


def load_data(nr_samples = 438, folder_path = 'priors_all_copy'):
    input_images = []
    target_pixels = []
    passed_pix = 0
    passed_img = 0
    dim = (192, 240) #width, heigth (ratio 0.8) 
    #dim = (384, 480)
    #dim = (960, 1200) #största testad hittills
    # dim = (1920, 2400)
    # dim = (2400, 3000) #största möjliga
    for filename in sorted(listdir(folder_path)):
        if "DS_Store" in filename:
            pass
        elif "mask" not in filename:
            try:
                #open image
                img_data = Image.open(folder_path + '/' + filename)
                img_rgb = img_data.convert('RGB')
                #open mask
                maskname = filename.split('.')[0] + "_mask.png"
                pix_data = Image.open(folder_path + '/' + maskname)
                #img_rgb.show()
                #input()
                for i in range(1): #2
                    #rotate image + add
                    rot_img = img_rgb.rotate(i*180, Image.NEAREST, expand = 1)
                    img_resized = rot_img.resize(dim)
                    data = np.array(img_resized)
                    print(data.shape)
                    Image.fromarray(data).show()
                    input()
                    new_filename = filename.split('.')[0] + "_" + str(i) + ".png"
                    input_images.append([new_filename,data])
                    #Image.fromarray(data,'RGB').show()
                    #input()
                    
                    #rotate mask + add
                    rot_pix = pix_data.rotate(i*180, Image.NEAREST, expand = 1)
                    data = np.array(rot_pix)
                    pixel_coord = np.where(data == True)
                    
                    #print(filename)
                    #print(pixel_coord)
                    #input()
                    img_size = rot_pix.size
                    new_maskname = filename.split('.')[0] + "_mask_" + str(i) + ".png"
                    target_pixels.append([new_maskname, (pixel_coord[1][0]/img_size[0])*dim[0], (pixel_coord[0][0]/img_size[1])*dim[1]])
                    
                    #mirror + add
                    '''
                    orig_mir = ImageOps.mirror(rot_img)
                    img_resized = orig_mir.resize(dim)
                    data = np.array(img_resized)
                    new_filename = filename.split('.')[0] + "_m_" + str(i) + ".png"
                    input_images.append([new_filename,data])

                    #mirror mask + add
                    mir_pix = ImageOps.mirror(rot_pix)
                    data = np.array(mir_pix)
                    pixel_coord = np.where(data == True)
                    img_size = rot_pix.size
                    new_maskname = filename.split('.')[0] + "_mask_m_" + str(i) + ".png"
                    target_pixels.append([new_maskname, (pixel_coord[1][0]/img_size[0])*dim[0], (pixel_coord[0][0]/img_size[1])*dim[1]])
                    '''
                    
            except (UnidentifiedImageError, NameError):
                passed_img += 1

    input_images = np.asarray(input_images)
    target_pixels = np.asarray(target_pixels)
    
    return input_images, target_pixels

#print(input_images.size)
#print(len(target_pixels))
#print(target_pixels)
#print(input_images)
#print(passed_pix)
#print(passed_img)

In [41]:
input_images, target_pixels = load_data()

print(input_images.shape)
print(target_pixels.shape)


(240, 192, 3)


NameError: name 'UnidentifiedImageError' is not defined

In [82]:
def gen_dataset(input_images, target_pixels): #generate dataset
    
    #make sure they're sorted properly
    #input_images = np.sort(input_images)
    #target_pixels = np.sort(target_pixels)
    
    #remove filenames (only keep input and target)
    X_img = np.squeeze(input_images[:,1])
    Y_pix = target_pixels[:,[1,2]]
    
    #shuffle same way
    #rng_state = np.random.get_state()
    #np.random.shuffle(X_img)
    #np.random.set_state(rng_state)
    #np.random.shuffle(Y_pix)
    
    #seed = np.random.randint(0, 100000)  
    #np.random.seed(seed)  
    #np.random.shuffle(X_img)  
    #np.random.seed(seed)  
    #np.random.shuffle(Y_pix)
    
    # divide into train and test 80/20: (or 90/10)
    split = int(len(X_img)*0.8)
    X_train_orig = X_img[0:split]
    X_test_orig = X_img[split+1:-1]
    Y_train_orig = Y_pix[0:split]
    Y_test_orig = Y_pix[split+1:-1]
    
    Y_pix = Y_pix.astype(float)

    
    return X_train_orig, Y_train_orig, X_test_orig, Y_test_orig

In [83]:
# load data
X_train_orig, Y_train, X_test_orig, Y_test = gen_dataset(input_images, target_pixels)

#normalize image vectors ? ------------- ??
X_train = X_train_orig/255
X_test = X_test_orig/255

#reformat image vectors
X_train = np.stack(X_train, axis=0) 
X_test = np.stack(X_test, axis=0)

In [67]:
Y_train

array([['172.2', '104.27884615384615'],
       ['187.2', '95.26442307692308'],
       ['161.1346153846154', '73.828125'],
       ['29.849999999999998', '107.95673076923077'],
       ['30.525000000000002', '75.21634615384615'],
       ['48.449999999999996', '111.49038461538461'],
       ['163.90384615384616', '154.04296875'],
       ['165.34615384615384', '105.703125'],
       ['128.85000000000002', '105.0'],
       ['126.63461538461539', '126.2109375'],
       ['41.82692307692307', '130.8984375'],
       ['13.73076923076923', '99.84375'],
       ['39.11538461538461', '69.31640625'],
       ['2.192307692307692', '87.5390625'],
       ['2.192307692307692', '124.27734375'],
       ['17.076923076923077', '128.84765625'],
       ['130.09615384615384', '93.046875'],
       ['152.325', '170.48076923076923'],
       ['171.075', '153.17307692307693'],
       ['43.55769230769231', '79.1015625'],
       ['48.92307692307692', '108.6328125'],
       ['29.19230769230769', '93.57421875'],
       ['42

In [90]:
print(X_train.shape)
print(X_test.shape)

print(X_train[0])
X_train, Y_train = shuffle_data(X_train, Y_train)
print(X_train[0])

(350, 240, 192, 3)
(86, 240, 192, 3)
[[[0.17647059 0.17647059 0.17647059]
  [0.16862745 0.16862745 0.16862745]
  [0.16862745 0.16862745 0.16862745]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.91764706 0.91764706 0.91764706]
  [0.89803922 0.89803922 0.89803922]
  [0.88627451 0.88627451 0.88627451]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.94901961 0.94901961 0.94901961]
  [0.9372549  0.9372549  0.9372549 ]
  [0.91372549 0.91372549 0.91372549]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 ...

 [[0.18431373 0.18431373 0.18431373]
  [0.16862745 0.16862745 0.16862745]
  [0.14117647 0.14117647 0.14117647]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.18823529 0.18823529 0.18823529]


In [85]:
350*4

1400

In [81]:
X_img = np.copy(X_train)
Y_pix = np.copy(Y_train)

vert = float(X_img.shape[1])
hort = float(X_img.shape[2])

img = X_img[0]
ver_img = np.flip(img, 0)
hor_img = np.flip(img, 1)
vh_img = np.flip(hor_img, 0)
ver_img = ver_img[np.newaxis]
hor_img = hor_img[np.newaxis]
vh_img = vh_img[np.newaxis]

pix = Y_pix[0].astype(float)
print(pix)
ver_pix = np.array([pix[0], vert - pix[1]])[np.newaxis]
hor_pix = np.array([hort - pix[0], pix[1]])[np.newaxis]
vh_pix = np.array([hort - pix[0], vert - pix[1]])[np.newaxis]

#add to arrays
X_img = np.append(X_img, ver_img, axis = 0)
X_img = np.append(X_img, hor_img, axis = 0)
X_img = np.append(X_img, vh_img, axis = 0)

Y_pix = np.append(Y_pix, ver_pix, axis = 0)                    
Y_pix = np.append(Y_pix, hor_pix, axis = 0)                    
Y_pix = np.append(Y_pix, vh_pix, axis = 0) 
                    
                    


[172.2        104.27884615]


In [80]:
im = Image.fromarray((X_img[-2] * 255).astype(np.uint8))
im.show()
print(Y_pix[-1])
print(Y_pix[-2])
print(Y_pix[0])

['19.80000000000001' '135.72115384615387']
['19.80000000000001' '104.27884615384615']
['172.2' '104.27884615384615']


In [58]:
Y_train[1]

array(['187.2', '95.26442307692308'], dtype='<U135')

In [469]:
#from numpy import savetxt
# save to csv file
#savetxt('X_384x480_rotmir.csv', input_images, delimiter=',', fmt='%s')
#savetxt('Y_384x480_rotmir.csv', target_pixels, delimiter=',', fmt='%s')

In [470]:
#im = Image.fromarray(np.uint8(X_test_orig[2]))
#im.show() #ok :))

## Data augmentation on training set

# Build model:

## Load pretrained resnet50 without top layer:

In [7]:
inshape = (480, 384, 3)
#inshape = (1200, 960, 3)
#inshape = (240, 192,  3) # 3 channels ??
base_model = ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=inshape, pooling=None) #weights='imagenet'

Instructions for updating:
Colocations handled automatically by placer.


/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


## Add top layers (regression):

In [8]:
X = GlobalMaxPooling2D()(base_model.output) #average vs max? global or not?
X = Dense(256, activation='relu')(X)
X = Dropout(0.7)(X)
output = Dense(2, activation = 'linear')(X) #hard_sigmoid?
model = Model(base_model.inputs, output)


# Train model on new data:

In [32]:

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all ResNet layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss='mean_squared_error') #Adam(learning_rate=0.01), loss='mean_squared_error') #optimizer: sgd? adam?

# train the model on the new data for a few epochs
#model.fit_generator(...)

BS = 35
EPOCHS = 2

# construct the training image generator for data augmentation
aug = ImageDataGenerator(shear_range=0.15)#, fill_mode="nearest") <-- default

# train the network
model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS), validation_data=(X_test, Y_test),
                    steps_per_epoch=len(X_train) // BS,
                    epochs=EPOCHS)


#model.fit_generator(batch_generator(X_train, y_train, 100, 1),
#                                  steps_per_epoch=len(X_train), 
#                                  epochs=2,
#                                  validation_data=batch_generator(X_test, y_test, 100, 0),
#                                  validation_steps=200,
#                                  verbose=1,
#                                  shuffle = 1)


#model.fit(X_train, Y_train, epochs = 10, batch_size = 1)


Epoch 1/2
45/45 [==============================] - 332s 7s/step - loss: 3553.4008 - val_loss: 11797.8959
Epoch 2/2
45/45 [==============================] - 323s 7s/step - loss: 2614.5941 - val_loss: 11903.8194


In [33]:
#EPOCHS = 5

#for i, layer in enumerate(base_model.layers):
#   print(i, layer.name)

print("Unfreeze last 2 ResNet-blocks:")

# we chose to train the top 2 resnet blocks, i.e. we will freeze
# the first 153 layers and unfreeze the rest:
for layer in model.layers[:153]:
   layer.trainable = False
for layer in model.layers[153:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model.compile(optimizer = 'adam', loss='mean_squared_error')

# we train our model again (this time fine-tuning the top 2 resnet blocks
# alongside the top Dense layers
#model.fit_generator(...)
#model.fit(X_train, Y_train, epochs = 5, batch_size = 1)
# train the network
model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS), validation_data=(X_test, Y_test), 
                    steps_per_epoch=len(X_train) // BS,
                    epochs=EPOCHS)
#model.fit_generator(...)


Unfreeze last 2 ResNet-blocks:
Epoch 1/2
45/45 [==============================] - 405s 9s/step - loss: 2010.8801 - val_loss: 7673.8384
Epoch 2/2
45/45 [==============================] - 383s 9s/step - loss: 1754.1104 - val_loss: 7483.6064


In [34]:

print("Unfreeze last 4 ResNet-blocks:")
EPOCHS = 2

for layer in model.layers[:129]:
   layer.trainable = False
for layer in model.layers[129:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model.compile(optimizer = 'adam', loss='mean_squared_error')

# we train our model again (this time fine-tuning the top 2 resnet blocks
# alongside the top Dense layers
#model.fit_generator(...)
#model.fit(X_train, Y_train, epochs = 5, batch_size = 1)
# train the network
model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS), validation_data=(X_test, Y_test),
                    steps_per_epoch=len(X_train) // BS,
                    epochs=EPOCHS)



Unfreeze last 4 ResNet-blocks:
Epoch 1/2
45/45 [==============================] - 448s 10s/step - loss: 2060.7034 - val_loss: 6182.5434
Epoch 2/2
45/45 [==============================] - 447s 10s/step - loss: 1787.9260 - val_loss: 6820.3754


# ↓ ↓ ↓ ↓ ↓ ↓ 

In [ ]:

print("Unfreeze all layers:")
EPOCHS = 10
BS = 10

for layer in model.layers:
   layer.trainable = True

aug = ImageDataGenerator(shear_range=0.15)#, fill_mode="nearest") <-- default

# we need to recompile the model for these modifications to take effect
model.compile(optimizer = 'adam', loss='mean_squared_error')

# we train our model again (this time fine-tuning the top 2 resnet blocks
# alongside the top Dense layers
#model.fit_generator(...)
#model.fit(X_train, Y_train, epochs = 5, batch_size = 1)
# train the network
model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS), validation_data=(X_test, Y_test), 
                    steps_per_epoch=len(X_train) // BS,
                    epochs=EPOCHS)

Unfreeze all layers:
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
39/39 [==============================] - 1246s 32s/step - loss: 13261.5399 - val_loss: 14588999631.2381
Epoch 2/10
39/39 [==============================] - 1243s 32s/step - loss: 7882.1827 - val_loss: 3250463.9405
Epoch 3/10
39/39 [==============================] - 1294s 33s/step - loss: 7823.4425 - val_loss: 58601.0467
Epoch 4/10
39/39 [==============================] - 1162s 30s/step - loss: 7347.8196 - val_loss: 18358.3207
Epoch 5/10
39/39 [==============================] - 1155s 30s/step - loss: 6603.2207 - val_loss: 20355.4426
Epoch 6/10
30/39 [======================>.......] - ETA: 5:28 - loss: 7208.6441

# Evaluate model:

In [ ]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [ ]:
model.summary()

In [ ]:
y = model.predict(X_test)

print(y)

In [ ]:
diff = abs(y.astype(float)-Y_test.astype(float))
print(diff[diff.sum(axis=1).argmin()])

# ^^^^^^

In [40]:
#scores = model.evaluate(X_test, Y_test, verbose=1) #predict vs evaluate
#??
#print("MSE = " + str(scores))

In [41]:
from math import sqrt
#sqrt(scores)

In [42]:
print(Y_test)

[['59.53846153846154' '119.765625']
 ['28.03846153846154' '154.04296875']
 ['54.224999999999994' '107.52403846153847']
 ['116.25' '54.84375']
 ['168.40384615384616' '91.34765625']
 ['31.44230769230769' '169.39453125']
 ['125.25' '115.78125']
 ['157.6153846153846' '115.6640625']
 ['91.84615384615385' '117.421875']
 ['26.596153846153847' '122.51953125']
 ['137.8269230769231' '108.45703125']
 ['32.36538461538461' '98.4375']
 ['171.1153846153846' '154.16015625']
 ['160.35000000000002' '106.94711538461539']
 ['165.34615384615384' '117.421875']
 ['18.975' '146.82692307692307']
 ['64.90384615384615' '108.22265625']
 ['54.11538461538462' '118.18359375']
 ['166.6730769230769' '113.14453125']
 ['25.26923076923077' '126.796875']
 ['159.60000000000002' '160.52884615384616']
 ['37.575' '110.40865384615385']
 ['1.3269230769230769' '115.95703125']
 ['9.975000000000001' '126.63461538461537']
 ['50.53846153846154' '104.0625']
 ['146.76923076923077' '76.40625']
 ['25.200000000000003' '87.11538461538461'

In [43]:
y_t = model.predict(X_train)

print(y_t)

[[58.84183  61.524162]
 [58.688805 61.262108]
 [58.858475 61.523613]
 ...
 [58.56729  61.014084]
 [58.863445 61.54534 ]
 [58.583897 61.063396]]


In [44]:
diff = abs(y_t.astype(float)-Y_train.astype(float))
diff.mean(axis=0)

array([59.14877768, 59.52691939])